<a href="https://colab.research.google.com/github/JayantBudania/my-first-pr/blob/main/Sudoku_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from typing import List, Tuple, Optional
import time

In [ ]:
Grid = List[List[int]]

In [ ]:
# Utility: pretty print the puzzle
def print_grid(g: Grid) -> None:
    for r in range(9):
        if r % 3 == 0 and r != 0:
            print("-" * 21)
        row = []
        for c in range(9):
            if c % 3 == 0 and c != 0:
                row.append("|")
            row.append(str(g[r][c]) if g[r][c] != 0 else ".")
        print(" ".join(row))
    print()

In [ ]:
# Check whether placing val at (r,c) is valid
def is_valid(g: Grid, r: int, c: int, val: int) -> bool:
    # row
    for j in range(9):
        if g[r][j] == val:
            return False
    # col
    for i in range(9):
        if g[i][c] == val:
            return False
    # 3x3 box
    br = (r // 3) * 3
    bc = (c // 3) * 3
    for i in range(br, br + 3):
        for j in range(bc, bc + 3):
            if g[i][j] == val:
                return False
    return True

In [ ]:
# Return list of candidate values for cell (r,c)
def candidates(g: Grid, r: int, c: int) -> List[int]:
    if g[r][c] != 0:
        return []
    used = set()
    used.update(g[r])  # row
    used.update(g[i][c] for i in range(9))  # col
    br = (r // 3) * 3
    bc = (c // 3) * 3
    for i in range(br, br + 3):
        for j in range(bc, bc + 3):
            used.add(g[i][j])
    return [v for v in range(1, 10) if v not in used]

In [ ]:
def select_unassigned_cell(g: Grid) -> Optional[Tuple[int,int]]:
    best = None
    best_count = 10
    for i in range(9):
        for j in range(9):
            if g[i][j] == 0:
                cand = candidates(g, i, j)
                if len(cand) < best_count:
                    best_count = len(cand)
                    best = (i, j)
                    if best_count == 1:
                        return best
    return best

In [ ]:
# Backtracking solver with MRV; returns True if solved
def solve_sudoku(g: Grid, stats: dict) -> bool:
    cell = select_unassigned_cell(g)
    if cell is None:
        return True  # solved
    r, c = cell
    cand = candidates(g, r, c)
    stats['calls'] += 1
    # try candidates in increasing order
    for val in cand:
        if is_valid(g, r, c, val):
            g[r][c] = val
            if solve_sudoku(g, stats):
                return True
            g[r][c] = 0
    return False

In [ ]:
# Helper to parse a puzzle given as 81-char string (0 or . for blanks) or 9x9 nested lists
def parse_puzzle(p):
    if isinstance(p, str):
        s = p.replace("\n", "").replace(" ", "")
        if len(s) != 81:
            raise ValueError("String puzzle must have 81 characters (use 0 or . for blanks).")
        g = []
        for i in range(9):
            row = []
            for j in range(9):
                ch = s[i*9 + j]
                if ch in "0.":
                    row.append(0)
                elif ch.isdigit():
                    row.append(int(ch))
                else:
                    raise ValueError("Invalid character in puzzle string.")
            g.append(row)
        return g
    elif isinstance(p, list):
        # assume 9x9 list
        if len(p) != 9 or any(len(row) != 9 for row in p):
            raise ValueError("List puzzle must be 9x9.")
        return [list(map(int, row)) for row in p]
    else:
        raise ValueError("Unsupported puzzle format")

In [ ]:
a = (
    "530070000"
    "600195000"
    "098000060"
    "800060003"
    "400803001"
    "700020006"
    "060000280"
    "000419005"
    "000080079"
)

b = (
    "000000000"
    "000003085"
    "001020000"
    "000507000"
    "004000100"
    "090000000"
    "500000073"
    "002010000"
    "000040009"
)


In [ ]:
def run_and_time(puzzle_input):
    grid = parse_puzzle(puzzle_input)
    print("Input puzzle:")
    print_grid(grid)
    stats = {'calls': 0}
    t0 = time.time()
    solved = solve_sudoku(grid, stats)
    t1 = time.time()
    if solved:
        print("Solved puzzle:")
        print_grid(grid)
    else:
        print("No solution found.")
    print(f"Time: {t1 - t0:.4f} s, recursive calls (select/unassigned): {stats['calls']}")

In [ ]:
if __name__ == "__main__":

    run_and_time(a)

Input puzzle:
5 3 . | . 7 . | . . .
6 . . | 1 9 5 | . . .
. 9 8 | . . . | . 6 .
---------------------
8 . . | . 6 . | . . 3
4 . . | 8 . 3 | . . 1
7 . . | . 2 . | . . 6
---------------------
. 6 . | . . . | 2 8 .
. . . | 4 1 9 | . . 5
. . . | . 8 . | . 7 9

Solved puzzle:
5 3 4 | 6 7 8 | 9 1 2
6 7 2 | 1 9 5 | 3 4 8
1 9 8 | 3 4 2 | 5 6 7
---------------------
8 5 9 | 7 6 1 | 4 2 3
4 2 6 | 8 5 3 | 7 9 1
7 1 3 | 9 2 4 | 8 5 6
---------------------
9 6 1 | 5 3 7 | 2 8 4
2 8 7 | 4 1 9 | 6 3 5
3 4 5 | 2 8 6 | 1 7 9

Time: 0.0017 s, recursive calls (select/unassigned): 51


In [ ]:
if __name__ == "__main__":
    # Run example 1 (easy)
    run_and_time(b)

Input puzzle:
. . . | . . . | . . .
. . . | . . 3 | . 8 5
. . 1 | . 2 . | . . .
---------------------
. . . | 5 . 7 | . . .
. . 4 | . . . | 1 . .
. 9 . | . . . | . . .
---------------------
5 . . | . . . | . 7 3
. . 2 | . 1 . | . . .
. . . | . 4 . | . . 9

Solved puzzle:
9 8 7 | 6 5 4 | 3 2 1
2 4 6 | 1 7 3 | 9 8 5
3 5 1 | 9 2 8 | 7 4 6
---------------------
1 2 8 | 5 3 7 | 6 9 4
6 3 4 | 8 9 2 | 1 5 7
7 9 5 | 4 6 1 | 8 3 2
---------------------
5 1 9 | 2 8 6 | 4 7 3
4 7 2 | 3 1 9 | 5 6 8
8 6 3 | 7 4 5 | 2 1 9

Time: 4.2162 s, recursive calls (select/unassigned): 58233
